In [1]:
%load_ext autoreload
%autoreload 2

%cd '..'

/dss/dsshome1/03/di93fup/polarization_reddit


In [2]:
from load.utils import (
    load_df_from_parquet,
    load_comments,
    save_df_as_parquet,
)

from preprocessing.utils import (
    tokenize_comment,
)

In [ ]:

import logging
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.FileHandler(f"data/logs/user_partisan_comments.log"),
        logging.StreamHandler(stream=sys.stdout),
    ],
)

In [3]:
from tqdm import tqdm

tqdm.pandas()


In [4]:
YEARS = [2015, 2016, 2017, 2018]

YEARS_STR = str(YEARS[0]) if len(YEARS) == 1 else f"{YEARS[0]}-{YEARS[-1]}"
START_MONTH = 1
STOP_MONTH = 12

logging.info(YEARS_STR)

In [5]:
user_party = load_df_from_parquet(
    file_name=f"user_party_{YEARS_STR}.parquet",
)[["author", "party"]]

user_party = user_party[user_party["party"].isin(["dem", "rep"])]


(619599, 2)
(537900, 2)


In [ ]:
for year in YEARS:
    comments_year = load_comments(
        years=year,
        start_month=START_MONTH,
        stop_month=STOP_MONTH,
        engine="pandas",
    )

    user_partisan_comments_year = comments_year.merge(
        user_party,
        on="author",
        how="inner",
    )

    logging.info("Tokenizing comments...")
    user_partisan_comments_year["tokens"] = (
        user_partisan_comments_year["body_cleaned"]
        .progress_apply(
            tokenize_comment,
        )
        .astype("string")
    )
    
    logging.info("Finish tokenizing comments")

    save_df_as_parquet(
        user_partisan_comments_year,
        target_file=f"user_partisan_comments_{year}.parquet",
    )
